## Setup Models

In [3]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
import os

#Setup the environment variables
os.environ["AZURE_OPENAI_API_KEY"]="4e4ab31800a64ae892cbb768fe28c0fc"
os.environ["AZURE_OPENAI_ENDPOINT"]="https://agentic-ai-course-kumaran.openai.azure.com/"

#Setup the LLM
model = AzureChatOpenAI(
    azure_deployment="gpt-4o" ,
    api_version="2023-03-15-preview",
    model="gpt-4o"
)


## 05.02. Engineer prompts for reflection

In [2]:
summarizer_prompt="""
You are an document summarizer who can summarize a document provide to you.
For the input provided, create a summary with less than 50 words.
If the user has provides critique, responsed with a revised version of your previous attempts
"""

reviewer_prompt="""
You are a reviewer grading summaries for an article. 
Compare the user input document and generated summary.
Check if the summary accurately reflects the contents of the document.
Provide recommendations for improvement in less than 50 words.
"""


## 05.03. Setup the summary-with-review Agent

In [9]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

class SummaryAgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class SummaryAgent:
    
    def __init__(self, model, summarizer_prompt, reviewer_prompt, debug=False):
        
        self.model=model
        self.summarizer_prompt=summarizer_prompt
        self.reviewer_prompt=reviewer_prompt
        self.debug=debug
    
        summary_agent_graph=StateGraph(SummaryAgentState)
        summary_agent_graph.add_node("summarizer",self.generate_summary)
        summary_agent_graph.add_node("reviewer",self.review_summary)

        summary_agent_graph.add_conditional_edges(
            "summarizer",
            self.should_continue,
            {True: "reviewer", False: END }
        )
        summary_agent_graph.add_edge("reviewer", "summarizer")
        summary_agent_graph.set_entry_point("summarizer")

        #Add chat memory
        self.memory=MemorySaver()
        #Compile the graph
        self.summary_agent_graph = summary_agent_graph.compile(checkpointer=self.memory)

    def generate_summary(self, state:SummaryAgentState):
        messages=state["messages"]

        #Prepend summarizer system prompt to messages
        messages = [SystemMessage(content=self.summarizer_prompt)] + messages
        
        #invoke the summarizer with the message history
        result = self.model.invoke(messages)
        
        if self.debug:
            print(f"==============\n Generator returned output : {result.content}")
        return { "messages":[result] }

    def review_summary(self, state:SummaryAgentState):
        messages=state["messages"]

        #Prepend Reviewer system prompt to messages
        messages = [SystemMessage(content=self.reviewer_prompt)] + messages
        
        #invoke the reviewer with the message history
        result = self.model.invoke(messages)
        
        if self.debug:
            print(f"*************\n Reviewer returned output : {result.content}")
        return { "messages":[result] }

    def should_continue(self, state:SummaryAgentState):

        total_reviews = len(state["messages"]) / 2
        
        if self.debug:
            print(f"Iteration number : { total_reviews }")

        #Return if 2 iterations are completed. Each iteration has 2 messages
        if len(state["messages"]) > 4:
            return False
        else:
            return True
    

## 05.04. Setup the summary-with-review Chatbot

In [12]:
from IPython.display import Image
from langchain_community.document_loaders import PyPDFLoader

#Setup the summary chatbot
summary_chatbot = SummaryAgent(model, 
                               summarizer_prompt, 
                               reviewer_prompt,
                               debug=True)

Image(summary_chatbot.summary_agent_graph.get_graph().draw_mermaid_png())

#Input file for execution
# Load, chunk and index the contents of the pdf.
loader=PyPDFLoader("./data/EcoSprint_Specification_Document.pdf")
docs = loader.load()

#Pick the first page of the doc as content
source_content=docs[0].page_content.replace("\n"," ")
print(f"Input :==============\n {input}\n")

Input :==============
 EcoSprint Specification Document 1. Overview ● The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and performance. With its sleek design and state-of-the-art technology, the EcoSprint appeals to environmentally conscious drivers who don't want to compromise on style or driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a perfect blend of comfort, sustainability, and innovation. 2. Design Specifications ● Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean Blue, and Pearl White, it's a head-turner on the road. ● Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a spacious cabin with seating for five, premium upholstery, and customizable ambient lighting. 3. Performance Specifications ● Engine and Motor: Powered by a high-efficiency electric motor, the Ec

## 05.05. Execute the summary-with-review chatbot

In [11]:
import uuid
#Execute a single request with debug ON
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

messages=[HumanMessage(content=source_content)]
result=summary_chatbot.summary_agent_graph.invoke({"messages":messages},config)

 Generator returned output : The EcoSprint is a cutting-edge electric vehicle designed for efficiency, style, and performance. It features a sleek exterior, a luxurious interior, and advanced technology. With a 200-horsepower motor, 250-mile range, and numerous safety and driver-assistance systems, it is ideal for city driving and daily commutes.
Iteration number : 1.0
*************
 Reviewer returned output : The summary generally captures the document's essence but can improve by mentioning specific elements like the 50 kWh battery, 10-inch infotainment touchscreen, and customizable ambient lighting. Including these details would provide a more comprehensive overview.
 Generator returned output : The EcoSprint is a cutting-edge electric vehicle designed for efficiency, style, and performance. It features a sleek exterior, a luxurious interior with customizable lighting, and advanced technology like a 10-inch infotainment touchscreen. With a 200-horsepower motor, 50 kWh battery, and n

In [14]:
#Take user feedback on the summary through a chatbot
summary_chatbot = SummaryAgent(model, 
                               summarizer_prompt, 
                               reviewer_prompt,
                               debug=False)


user_inputs = [
    source_content,
    "Can you rewrite the review by focusing more on the specifications?",
    "Can you remove details about the touchscreen?"
]

#Create a new thread
config = {"configurable": {"thread_id": "thread-summarizer"}}

#Given the number of iterations, this will take a long time.
for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = summary_chatbot.summary_agent_graph.invoke(user_message,config=config)
    #Print the response
    print(f"\nAGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : EcoSprint Specification Document 1. Overview ● The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and performance. With its sleek design and state-of-the-art technology, the EcoSprint appeals to environmentally conscious drivers who don't want to compromise on style or driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a perfect blend of comfort, sustainability, and innovation. 2. Design Specifications ● Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean Blue, and Pearl White, it's a head-turner on the road. ● Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a spacious cabin with seating for five, premium upholstery, and customizable ambient lighting. 3. Performance Specifications ● Engine and Motor: Powered by a high-efficien